In [3]:
import torch
from mambapy.mamba import Mamba, MambaConfig
from torchinfo import summary
from c66 import pp

# B, L, D = 7, 64, 16
B, L, D = 7, 201, 286
pp(B,L,D)

config = MambaConfig(d_model=D, n_layers=1, use_cuda=False)
model = Mamba(config)

B: 7
L: 201
D: 286


In [4]:
model.layers[0].mixer.A_log

Parameter containing:
tensor([[0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        ...,
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726],
        [0.0000, 0.6931, 1.0986,  ..., 2.6391, 2.7081, 2.7726]],
       requires_grad=True)

In [5]:
model.layers[0].mixer.D

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.

In [1]:
import torch
from mambapy.mamba import Mamba, MambaConfig
from torchinfo import summary
from c66 import pp

# B, L, D = 7, 64, 16
B, L, D = 7, 201, 286
pp(B,L,D)

config = MambaConfig(d_model=D, n_layers=1, use_cuda=False)
model = Mamba(config)
# .to("cuda")

x = torch.randn(B, L, D)
# .to("cuda")
y = model(x)

assert y.shape == x.shape
pp(x.shape)

B: 7
L: 201
D: 286
------------------------------
In ResidualBlock
x.shape: torch.Size([7, 201, 286])
self.norm(x).shape: torch.Size([7, 201, 286])
------------------------------
------------------------------
In MambaBlock
torch.Size([7, 572, 201])
torch.Size([7, 572, 201])
B, L, ED, N, dt_rank: 7 201 572 16 18
x.shape, delta.shape, A.shape, B.shape, C.shape, z.shape:
torch.Size([7, 201, 572]) torch.Size([7, 201, 572]) torch.Size([572, 16]) torch.Size([7, 201, 16]) torch.Size([7, 201, 16]) torch.Size([7, 201, 572])
self.selective_scan
------------------------------
x.shape: torch.Size([7, 201, 286])


In [4]:
print(model)

Mamba(
  (layers): ModuleList(
    (0): ResidualBlock(
      (mixer): MambaBlock(
        (in_proj): Linear(in_features=286, out_features=1144, bias=False)
        (conv1d): Conv1d(572, 572, kernel_size=(4,), stride=(1,), padding=(3,), groups=572)
        (x_proj): Linear(in_features=572, out_features=50, bias=False)
        (dt_proj): Linear(in_features=18, out_features=572, bias=True)
        (out_proj): Linear(in_features=572, out_features=286, bias=False)
      )
      (norm): RMSNorm()
    )
  )
)


In [ ]:
print(model.layers[0].mixer)

MambaBlock(
  (in_proj): Linear(in_features=286, out_features=1144, bias=False)
  (conv1d): Conv1d(572, 572, kernel_size=(4,), stride=(1,), padding=(3,), groups=572)
  (x_proj): Linear(in_features=572, out_features=50, bias=False)
  (dt_proj): Linear(in_features=18, out_features=572, bias=True)
  (out_proj): Linear(in_features=572, out_features=286, bias=False)
)


In [4]:
summary_str = summary(model, input_size=[(B,L,D)], depth=5, col_names=("input_size", "output_size", "num_params"), verbose=0)
print(summary_str)

torch.Size([7, 201, 286])
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
Mamba                                    [7, 201, 286]             [7, 201, 286]             --
├─ModuleList: 1-1                        --                        --                        --
│    └─ResidualBlock: 2-1                [7, 201, 286]             [7, 201, 286]             --
│    │    └─RMSNorm: 3-1                 [7, 201, 286]             [7, 201, 286]             286
│    │    └─MambaBlock: 3-2              [7, 201, 286]             [7, 201, 286]             20,592
│    │    │    └─Linear: 4-1             [7, 201, 286]             [7, 201, 1144]            327,184
│    │    │    └─Conv1d: 4-2             [7, 572, 201]             [7, 572, 204]             2,860
│    │    │    └─Linear: 4-3             [7, 201, 572]             [7, 201, 50]              28,600
│    │    │    └─Linear: 4-4             [7, 201, 572]             [7, 201, 286]        